# Analyses des fichiers 001MoDe_R1.csv & 001MoDe_R1.marker.csv

Ce document détaille les formules mathématiques utilisées pour transformer les données brutes de position (`001MoDe_R1.csv`) en métriques de performance et de précision (`001MoDe_R1.marker.csv`), basées sur le modèle de la Loi de Fitts pour les tâches de direction (*steering*).

## 1. Paramètres de la Tâche (Extraits de l'En-tête)

Les valeurs suivantes sont nécessaires pour tous les calculs et sont extraites de l'en-tête du fichier `.csv` :

| Variable | Symbole | Valeur (Exemple) | Description |
| :--- | :--- | :--- | :--- |
| `centerX` | $C_x$ | 552.0 | Coordonnée X du centre de la tâche. |
| `centerY` | $C_y$ | 330.0 | Coordonnée Y du centre de la tâche. |
| `externalRadius` | $R_{\text{ext}}$ | 250.0 | Rayon externe du couloir cible. |
| `internalRadius` | $R_{\text{int}}$ | 170.0 | Rayon interne du couloir cible. |
| `cycleDuration` | $T_{\text{rec}}$ | 20.0 (s) | Durée de chaque cycle d'enregistrement. |

## 2. Métriques Géométriques et Précision

Ces formules permettent de calculer les métriques de position et de variabilité à partir des coordonnées brutes du curseur $(x_i, y_i)$ pour un enregistrement donné (Rec00X).

### A. Distance Instantanée au Centre ($R_i$)

La distance du curseur au centre de la tâche à l'échantillon $i$.

$$R_i = \sqrt{(x_i - C_x)^2 + (y_i - C_y)^2}$$

### B. Amplitude Effective ($R_e$)

**`Re` (pixel)** : La distance radiale moyenne effectivement parcourue par le curseur.

$$R_e = \frac{1}{N} \sum_{i=1}^{N} R_i$$

### C. Largeur Effective de la Cible ($T_e$)

**`Te` (pixel)** : La tolérance effective autour de la cible, basée sur l'écart-type radial ($\sigma_R$).

$$T_e = 4.133 \times \sigma_R$$
Avec :
$$\sigma_R = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (R_{i} - R_{e})^2}$$

### D. Taux d'Erreur (`error`)

**`error` (%)** : Le pourcentage du temps passé en dehors du couloir cible ($[R_{\text{int}}, R_{\text{ext}}]$).

$$\text{error} = \frac{\sum_{i=1}^{N} \Delta t_i \cdot \mathbb{I}(R_i \notin [R_{\text{int}}, R_{\text{ext}}])}{T_{\text{rec}}} \times 100$$
*(Où $\Delta t_i$ est l'intervalle de temps et $\mathbb{I}(\dots)$ est la fonction indicatrice.)*

## 3. Métriques de Performance et Débit d'Information

Ces formules utilisent les métriques géométriques et le temps pour calculer la performance selon la Loi de Fitts.

### E. Nombre de Tours ($n_{\text{Laps}}$)

**`nLaps` (lap)** : Calculé en comptant le nombre de révolutions complètes de $360^\circ$ de l'angle $\theta_i$ (angle entre le curseur et le centre) :
$$\theta_i = \text{atan2}(y_i - C_y, x_i - C_x)$$
Le nombre de tours est déterminé par le nombre de fois où l'angle $\theta_i$ passe de $\approx \pi$ à $\approx -\pi$ (ou inversement), après ajustement pour le saut de phase $2\pi$.

### F. Temps de Mouvement par Tour ($\text{MT/lap}$)

**`MT/lap` (s/lap)** : Le temps moyen mis pour compléter un tour.

$$\text{MT/lap} = \frac{T_{\text{rec}}}{n_{\text{Laps}}}$$

### G. Indice de Difficulté Effective ($\text{ID}_e/\text{lap}$)

**`IDe/lap` (bit/lap)** : La difficulté de la tâche, basée sur la circonférence effective ($2 \pi R_e$) et la largeur effective ($T_e$).

$$\text{ID}_e/\text{lap} = \log_2 \left( \frac{2 \pi R_e}{T_e} \right)$$


### H. Taux de Traitement de l'Information Effectif ($\text{IP}_e$)

**`IPe` (bit/s)** : Le débit de performance (Throughput), la métrique principale de la Loi de Fitts.

$$\text{IP}_e = \frac{\text{ID}_e/\text{lap}}{\text{MT/lap}}$$

### I. Coefficient de Régression ($B_e$)

**`Be` (double)** : Le coefficient $B$ dans l'équation de Fitts ($MT = A + B \cdot ID$).

* **Note Importante :** Le coefficient $B_e$ est traditionnellement obtenu par une **régression linéaire** sur les données de plusieurs tâches avec différents $ID$.
* Étant donné que votre analyse porte sur un seul type de tâche (CircularTarget), la valeur $B_e$ présente dans le fichier `.marker.csv` est généralement soit :
    1.  Le coefficient $B$ obtenu à partir de la régression effectuée sur **tous les enregistrements** (Rec001 à Rec005) du participant.
    2.  Un coefficient de biais ou une mesure de régularité du mouvement impliquant des dérivées (vitesse, accélération), spécifique à l'implémentation de **MouseReMoCo**.
* **Pour la reproduction :** Vous devrez reproduire la valeur fournie, soit en intégrant les données de tous les cycles dans une régression $MT \sim ID_e$ pour obtenir $B$ (si vous avez plusieurs conditions), soit en utilisant la formule spécifique à la régularité du mouvement si elle est documentée.

## 4. Reproduction du Fichier Marker

L'étape finale consiste à écrire les valeurs calculées ci-dessus dans le format tabulaire du fichier `.marker.csv` pour chaque cycle d'enregistrement (Rec001 à Rec005) afin de valider vos résultats.